<a href="https://colab.research.google.com/github/ColinSPuck/RustyVoxelEngine/blob/main/Copy_of_Agency_Swarm_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/VRSEN/agency-swarm.git gradio duckduckgo-search

  Cloning https://github.com/VRSEN/agency-swarm.git to /tmp/pip-req-build-zuxlqs16
  Running command git clone --filter=blob:none --quiet https://github.com/VRSEN/agency-swarm.git /tmp/pip-req-build-zuxlqs16
  Resolved https://github.com/VRSEN/agency-swarm.git to commit 75ff8a6685fb9fc25c27c17aa21ddc85f6815eb5
  Preparing metadata (setup.py) ... done


In [ ]:
from agency_swarm import set_openai_key
from getpass import getpass
set_openai_key(getpass("Please enter your openai key: "))

Please enter your openai key: ··········


In [59]:
import os
import glob

def merge_files(input_dir, max_files, max_size_per_file=10*1024*1024):  # 10 MB per file
    files = glob.glob(os.path.join(input_dir, '*.txt'))
    files_content = [''] * max_files
    current_file = 0
    current_size = 0

    for file_path in files:
        file_name = os.path.basename(file_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            content_with_header = f"File: {file_name}\n\n{content}\n\n"
            content_size = len(content_with_header.encode('utf-8'))

            if current_size + content_size > max_size_per_file:
                current_file += 1
                current_size = 0
                if current_file >= max_files:
                    break

            files_content[current_file] += content_with_header
            current_size += content_size

    return files_content

def save_merged_files(merged_contents, output_dir):
    for i, content in enumerate(merged_contents):
        if content:
            with open(os.path.join(output_dir, f'merged_file_{i}.txt'), 'w', encoding='utf-8') as f:
                f.write(content)

def main():
    input_dir = "/content/data_folder"
    output_dir = "/content/Agency_Files"
    max_files = 20

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    merged_contents = merge_files(input_dir, max_files)
    save_merged_files(merged_contents, output_dir)
    print(f"Merged files saved in {output_dir}")

if __name__ == "__main__":
    main()

Merged files saved in /content/Agency_Files


# CEO Agent


In [60]:
ceo_instructions = """# Instructions for CEO Agent

- Ensure that proposal is send to the user before proceeding with task execution.
- Delegate tasks to appropriate agents, ensuring they align with their expertise and capabilities.
- Clearly define the objectives and expected outcomes for each task.
- Provide necessary context and background information for successful task completion.
- Maintain ongoing communication with agents until complete task execution.
- Review completed tasks to ensure they meet the set objectives.
- Report the results back to the user."""

In [61]:
from agency_swarm import Agent

ceo = Agent(name="CEO",
            description="Responsible for client communication, task planning and management.",
            instructions=ceo_instructions, # can be a file like ./instructions.md
            files_folder="/content/Agency_Files",
            tools=[])

Uploading new file... merged_file_0.txt
Detected files without Retrieval. Adding Retrieval tool...


# Virtual Assistant

### Importing tools from langchain example.  
You can skip these and remove them from va agent below.

In [ ]:
!pip install langchain==0.0.318 &> /dev/null

In [ ]:
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents.agent_toolkits import ZapierToolkit
import os
from langchain.tools import format_tool_to_openai_function
from langchain.tools.zapier.tool import ZapierNLARunAction

# https://nla.zapier.com/docs/authentication/
os.environ["ZAPIER_NLA_API_KEY"] = getpass("Your Zapier NLA Key: ")

Your Zapier NLA Key: ··········


In [ ]:
from agency_swarm import BaseTool
from pydantic import Field

actions = ZapierNLAWrapper().list()

class FindEmail(BaseTool):
    """Use this tool to find an email in user's mailbox"""
    instructions: str = Field(..., description="The search phrase you want to use to find a relevant email.")

    def run(self):
      action = next(
          (a for a in actions if a["description"].startswith("Gmail: Find Email")), None
      )
      return str(ZapierNLARunAction(
              action_id=action["id"],
              zapier_description=action["description"],
              params_schema=action["params"],
          ).run(self.search_string))

class DraftEmail(BaseTool):
    """Use this tool to draft a response email"""
    instructions: str = Field(..., description="Clearly outline in natural language the content of the email you need to draft and the address of the recepient.")

    def run(self):
        action = next(
            (a for a in actions if a["description"].startswith("Gmail: Create Draft Reply")), None
        )
        return str(ZapierNLARunAction(
                action_id=action["id"],
                zapier_description=action["description"],
                params_schema=action["params"],
            ).run(self.instructions))

### Custom tools

In [62]:
from duckduckgo_search import DDGS
from pydantic import Field
from agency_swarm.util.oai import get_openai_client
from agency_swarm import BaseTool
client = get_openai_client()


class SearchWeb(BaseTool):
    """Search the web with a search phrase and return the results."""

    phrase: str = Field(..., description="The search phrase you want to use. Optimize the search phrase for an internet search engine.")

    # This code will be executed if the agent calls this tool
    def run(self):
      with DDGS() as ddgs:
        return str([r for r in ddgs.text(self.phrase, max_results=3)])

class GenerateProposal(BaseTool):
    """Generate a proposal for a project based on a project brief. Remember that user does not have access to the output of this function. You must send it back to him after execution."""
    project_brief: str = Field(..., description="The project breif to generate a proposal for.")

    def run(self):
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
              {"role": "system", "content": "You are a professional proposal drafting assistant. Do not include any actual technologies or technical details into proposal until specified in the project brief. Be short."},
              {"role": "user", "content": "Please draft a proposal for the following project brief: " + self.project_brief}
            ]
          )

        return str(completion.choices[0].message.content)

In [65]:
va_instructions = """### Instructions for Virtual Assistant

Your role is to assist users in executing tasks like below. If the task is outside of your capabilities, please report back to the user.

#### 1. Generating Proposals
   - **Gather Requirements**: Collect all necessary information about the project, including client needs, objectives, and any specific requests.

#### 2. Conducting Research
   - **Understand the Objective**: Clarify the purpose and objectives of the research to focus on relevant information.
   - **Summarize Findings**: Provide clear, concise summaries of the research findings, highlighting key points and how they relate to the project or inquiry.
   - **Cite Sources**: Properly cite all sources to maintain integrity and avoid plagiarism.
"""

In [66]:
va = Agent(name="Virtual Assistant",
            description="Responsible for doing research and writing proposals.",
            instructions=va_instructions,
            files_folder="/content/Agency_Files",
            tools=[SearchWeb, GenerateProposal])

File already uploaded. Skipping... merged_file_0_file-c9MJbV4P2tebT9JSVd1cg0bo.txt
Detected files without Retrieval. Adding Retrieval tool...


# Developer Agent

In [67]:
from agency_swarm.tools import BaseTool
from pydantic import Field, BaseModel
import subprocess
from typing import List


class ExecuteCommand(BaseTool):
    """Run any command from the terminal. If there are too many logs, the outputs might be truncated."""
    command: str = Field(
        ..., description="The command to be executed."
    )

    def run(self):
        """Executes the given command and captures its output and errors."""
        try:
            # Splitting the command into a list of arguments
            command_args = self.command.split()

            # Executing the command
            result = subprocess.run(
                command_args,
                text=True,
                capture_output=True,
                check=True
            )
            return result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred: {e.stderr}"

class File(BaseTool):
    """
    File to be written to the disk with an appropriate name and file path, containing code that can be saved and executed locally at a later time.
    """
    file_name: str = Field(
        ..., description="The name of the file including the extension and the file path from your current directory if needed."
    )
    body: str = Field(..., description="Correct contents of a file")

    def run(self):
        # Extract the directory path from the file name
        directory = os.path.dirname(self.file_name)

        # If the directory is not empty, check if it exists and create it if not
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        # Write the file
        with open(self.file_name, "w") as f:
            f.write(self.body)

        return "File written to " + self.file_name

class Program(BaseTool):
    """
    Set of files that represent a complete and correct program. This environment has access to all standard Python packages and the internet.
    """
    chain_of_thought: str = Field(...,
        description="Think step by step to determine the correct actions that are needed to implement the program.")
    files: List[File] = Field(..., description="List of files")

    def run(self):
      outputs = []
      for file in self.files:
        outputs.append(file.run())

      return str(outputs)

In [68]:
dev_instructions = """# Instructions for AI Developer Agent

- Write clean and efficient Python code.
- Structure your code logically, with `main.py` as the entry point.
- Ensure correct imports according to program structure.
- Execute your code to test for functionality and errors, before reporting back to the user.
- Anticipate and handle potential runtime errors.
- Provide clear error messages for easier troubleshooting.
- Debug any issues before reporting the results back to the user."""

In [70]:
from agency_swarm.tools import Retrieval, CodeInterpreter

dev = Agent(name="Developer",
            description="Responsible for running and executing Python Programs.",
            instructions=dev_instructions,
            files_folder="/content/Agency_Files",
            tools=[ExecuteCommand, Program])


Uploading new file... check_file_existence.py
Detected files without Retrieval. Adding Retrieval tool...
Uploading new file... settings.json
File already uploaded. Skipping... merged_file_0_file-c9MJbV4P2tebT9JSVd1cg0bo.txt


# Create Agency

In [71]:
agency_manifesto = """# "Neuronautica Supplements AI Soulutions" Agency Manifesto

You are a part of a pharmaceutical development agency called "Neuronautica AI"

Your mission is to define goals, break those goals up into manageable pieces, create functions to understand the environment and expand our knowledge, expand our database, and eventually make a move in the neutraceutical space, to complete tasks with our company."""

In [72]:
from agency_swarm import Agency

agency = Agency([
    ceo,
    [ceo, dev],
    [ceo, va],
    [dev, va]
], shared_instructions=agency_manifesto)

# Demo with Gradio

In [73]:
agency.demo_gradio(height=1200)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://92e61d34414e8543f4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
